## Create API calls from two different sources - Foursquare, Yelp, (Google - stretch) for your Places of Interest (POI)

### 1. Libraries imported for API calls

In [1]:
# import modules required

import pandas as pd
import os
import json
from pandas.io.json import json_normalize
import requests
import urllib

from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode

### 2. Foursqare API Call

##### Functions for API calls and parameter inputs

In [2]:
# lat/long for downtown Kamloops
# radius set to 1,000 meters (1km) to align with the maximum in Yelp

latitude = '50.67456895939129'
longitude =  '-120.32748594917784'
restaurant_categories = '&categories=13000'
outdoors_categories = '&categories=16000'
location_name = 'query=Kamloops'

location = 'll='+latitude+'%2C'+longitude

def fsq_response_call(location, category):
    """Given your API_KEY, send a GET request to the API.
    Args:
        location (str): combined string of latitude and longitude of location (Kamloops currently).
        category (str): ID of category to be used for API call, 5 digit integer
    Output:
        status_code: printed to display status of API call (200 = success)
    Returns:
        dict: The JSON response from the request.
    """

    limit = '&limit=50'
    radius = '&radius=1000'
    rating = '&fields=rating%2Cname%2Cdistance%2Clocation%2Ccategories%2Cstats'

    append = location+category+limit+radius+rating
    url = "https://api.foursquare.com/v3/places/search?"+append
    headers = {"accept": "application/json",
                "Authorization": os.environ["fsqr_api_key"]}
    response = requests.get(url, headers=headers)
    print(response.status_code)
    return response.json()

In [3]:
# API calls for restaurants and outdoor items

fsq_pd_rsts = fsq_response_call(location,restaurant_categories)
fsq_pd_outd = fsq_response_call(location,outdoors_categories)

200
200


#### Functions for parsing and handling API calls

In [4]:
# running API calls to put into payload variables for alteration, do not need to run API calls constantly with this method
# normalizes the payload into a dataframe, uses a lambda function to pull out the category id, then drops the category column
# final result is clean data with a category_id foreign key
# consolidates data into one table

fsq_field_filters = ['categories', 'location.address', 'location.country', 'location.formatted_address', \
'location.locality', 'location.neighborhood', 'location.postcode', 'location.region', \
'location.cross_street', 'location.address_extended', 'stats.total_photos', 'stats.total_ratings', 'stats.total_tips']

fsq_field_filters2 = ['categories', 'location.address', 'location.country', 'location.formatted_address', \
'location.locality', 'location.neighborhood', 'location.postcode', 'location.region', \
'location.cross_street', 'location.address_extended']

def fsq_payload_parser(api_payload, fields_to_remove):
    """Takes inputs from payload_categories for processing and final parsing
    Args:
        api_payload (dict): JSON format API call document
        fields_to_remove (list): List with all of the fields to be removed from the JSON DataFrame
    Returns:
        category_list (DataFrame): Category listing of all POI captured in API call, only type listed
        clean_payload (DataFrame): Cleaned API DataFrame with values to go into SQL DB
    """
    
    api_filter = list(api_payload.keys())[0]
    json_payload = pd.json_normalize(api_payload[api_filter])
    cat_filter = list(json_payload)[0]

    category_list = pd.json_normalize(api_payload[api_filter], record_path=cat_filter)

    json_payload['category_id'] = json_payload['categories'].apply(lambda x: x[0]['id'])
    if api_payload == fsq_pd_rsts:
        json_payload['total_ratings'] = json_payload['stats.total_ratings']
    clean_payload = json_payload.drop(fields_to_remove, axis=1)    
    return clean_payload, category_list

    
    

In [5]:
# function calls and data consolidation into one table

fsq_rests, fsq_rest_locs = fsq_payload_parser(fsq_pd_rsts, fsq_field_filters)
fsq_outdr, fsq_outd_locs = fsq_payload_parser(fsq_pd_outd, fsq_field_filters2)

fsq_cats = pd.concat([fsq_rest_locs, fsq_outd_locs], ignore_index=True).drop_duplicates()
fsq_locations = pd.concat([fsq_rests, fsq_outdr], ignore_index=True)

fsq_cats = fsq_cats.drop(['icon.suffix', 'icon.prefix'],axis=1)



In [6]:
fsq_locations.head()
fsq_cats.head()

,id,name
0,13065,Restaurant
1,13003,Bar
2,13029,Brewery
6,13027,Bistro
8,13199,Indian Restaurant


### 3. Yelp API Call

#### Functions for API calls and parameter inputs

In [7]:
# variables for Yelp API calls
# yelp filters on search results are not organized in any reasonable matter

# location metrics used for API call
# latitude = '50.67456895939129'
# longitude =  '-120.32748594917784'
location_name = 'Kamloops, BC'

# search term adjustments, different functions use these two variables
search_term_rest = 'restaurants'
search_term_outdoors = 'outdoors'

# API call parameters
yelp_api_host = 'https://api.yelp.com'
yelp_search_path = '/v3/businesses/search'
yelp_api_key = os.environ['yelp_api_key']
yelp_client_ID = os.environ['yelp_client_ID']
yelp_bearer = 'Bearer_API_KEY'
yelp_search_limit = 50


In [8]:
# Yelp Business search does not return any businesses without reviews
# defining Yelp API call, credit given to yelp-fusion public Git

def yelp_request_call(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """

    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))
    response = requests.request('GET', url, headers=headers, params=url_params)
    print(response.status_code)
    return response.json()

In [9]:
# defining Yelp search which returns the API call function
# credit given to yelp-fusion public Git

def yelp_search(api_key, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location_name.replace(' ', '+'),
        'limit': yelp_search_limit
    }
    return yelp_request_call(yelp_api_host, yelp_search_path, api_key, url_params=url_params)

In [10]:
# API calls for restaurants and outdoor locations

yelp_pd_rest = yelp_search(yelp_api_key, search_term_rest, location_name)
yelp_pd_outd = yelp_search(yelp_api_key, search_term_outdoors, location_name)

Querying https://api.yelp.com/v3/businesses/search ...
200
Querying https://api.yelp.com/v3/businesses/search ...
200


#### Functions for parsing and handling API calls

In [11]:
# running API calls to put into payload variables for alteration, do not need to run API calls constantly with this method
# normalizes the payload into a dataframe, uses a lambda function to pull out the category id, then drops the category column
# final result is clean data with a category_id foreign key

yelp_field_filter = ['alias', 'image_url', 'is_closed', 'url', 
       'categories', 'transactions', 'price', 'phone',
       'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address']

def yelp_payload_parser(api_payload, fields_to_remove):
    """Takes inputs from payload_categories for processing and final parsing
    Args:
        api_payload (dict): JSON format API call document
        fields_to_remove (list): List with all of the fields to be removed from the JSON DataFrame
    Returns:
        category_list (DataFrame): Category listing of all POI captured in API call, only type listed
        clean_payload (DataFrame): Cleaned API DataFrame with values to go into SQL DB
    """
    api_filter = list(api_payload.keys())[0]
    json_payload = pd.json_normalize(api_payload[api_filter])
    cat_filter = list(json_payload)[7]
    
    category_list = pd.json_normalize(api_payload[api_filter], record_path=cat_filter)
    
    json_payload['category_id'] = json_payload['categories'].apply(lambda x: x[0]['alias'])
    clean_payload = json_payload.drop(fields_to_remove, axis=1)
    return clean_payload, category_list



In [12]:
# function calls and data consolidation into one table

yelp_rest_locs, yelp_rest_cats = yelp_payload_parser(yelp_pd_rest, yelp_field_filter)
yelp_outd_locs, yelp_outd_cats = yelp_payload_parser(yelp_pd_outd, yelp_field_filter)

yelp_cats = pd.concat([yelp_rest_cats, yelp_outd_cats], ignore_index=True).drop_duplicates()
yelp_locations = pd.concat([yelp_rest_locs, yelp_outd_locs], ignore_index=True)

In [13]:
yelp_locations.head()
yelp_cats.head()

,alias,title
0,brewpubs,Brewpubs
1,burgers,Burgers
2,pizza,Pizza
3,diners,Diners
4,steak,Steakhouses


### 4. Database prepartion

#### Libraries imported, tables to create

In [14]:
import sqlite3
conn = sqlite3.connect('miniproject2.db')
c = conn.cursor()

#### Creation of SQL Tables

In [15]:
# To identify which columns need to be created in the SQL tables

def table_creation(data_tables, table_names):
    conn = sqlite3.connect('miniproject2.db')
    n = 0
    c = conn.cursor()
    for table in data_tables:
        new_columns = set()
        for sql_column in table:
            if 'stats.' in sql_column:
                column = sql_column.replace('stats.','')
                new_columns.add(column)
            elif 'icon.' in sql_column:
                column = sql_column.replace('icon.','')
                new_columns.add(column)
            else:
                column = sql_column
                new_columns.add(column)
        column_names = tuple(new_columns)
        c.execute(f'CREATE TABLE IF NOT EXISTS {table_names[n]} {column_names}')
        conn.commit
        # table.to_sql(f"'{table_names[n]}'", conn, if_exists='replace', index = False)
        n += 1

#### Loading data into tables

In [16]:
tables_to_make = [fsq_locations, fsq_cats, yelp_locations, yelp_cats]
table_name_list = ['fsq_loc', 'fsq_cats', 'yelp_loc', 'yelp_cats']

table_creation(tables_to_make, table_name_list)

yelp_cats.to_sql('yelp_cats', conn, if_exists='replace', index = False)
fsq_cats.to_sql('fsq_cats', conn, if_exists='replace', index = False)

yelp_locations.to_sql('yelp_loc', conn, if_exists='replace', index = False)
fsq_locations.to_sql('fsq_loc', conn, if_exists='replace', index = False)

61

In [19]:
sqlite3 miniproject2.db.schema > database_schema.txt

SyntaxError: invalid syntax (812656140.py, line 1)